In [1]:
import numpy as np
import time
import socket

# Define the IP address and port for the simulation
ip_address = "127.0.0.1"
port = 1234  # Replace with the port number used by your simulation

# Setup socket connection
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((ip_address, port))
print("Connected to simulation at {}:{}".format(ip_address, port))

# Load the corner and drop points
corner_points = np.load('corner_points.npz')['arr_0']
drop_points = np.load('drop_points.npz', allow_pickle=True)['arr_0'].item()

# Function to calculate the correct height offset based on the battery type
def get_height_offset(battery_type):
    if battery_type == 'AA':
        return 1.0  # 1 cm for AA
    elif battery_type == '9V':
        return 2.0  # 2 cm for 9V
    elif battery_type == 'D':
        return 4.0  # 4 cm for D
    else:
        return 0.0  # Default

# Function to send command to the robot
def send_command(command):
    sock.sendall(command.encode())
    print(f"Command sent: {command.strip()}")

# Function to move the robot to a specific position
def move_to_position(x, y, z):
    command = f"G01 X{x:.2f} Y{y:.2f} Z{z:.2f}\n"  # G-code command format
    send_command(command)

# Function to turn the electromagnet ON or OFF
def electromagnet(state):
    if state == "ON":
        command = "M3 S100\n"  # Example command to turn on the electromagnet
        send_command(command)
        print("Turning electromagnet ON")
    elif state == "OFF":
        command = "M5\n"  # Example command to turn off the electromagnet
        send_command(command)
        print("Turning electromagnet OFF")

# Function to sort a detected battery
def sort_battery(detection):
    battery_type = detection['battery_type']
    x, y = detection['coordinates']

    # Adjust Z position based on battery type
    z_offset = get_height_offset(battery_type)
    
    # Move to the battery position
    move_to_position(x, y, 10 + z_offset)  # Example Z-axis height adjustment
    time.sleep(3)  # Wait 3 seconds
    
    # Turn on the electromagnet to pick the battery
    electromagnet("ON")
    time.sleep(3)  # Wait another 3 seconds
    
    # Move to the drop point for the detected battery type
    drop_x, drop_y, drop_z = drop_points[battery_type]
    move_to_position(drop_x, drop_y, drop_z)
    time.sleep(3)  # Wait 3 seconds
    
    # Turn off the electromagnet to release the battery
    electromagnet("OFF")
    time.sleep(3)  # Wait 3 seconds

# Example detected batteries (from the detection script)
detected_batteries = [
    {'battery_type': 'AA', 'coordinates': (100, 200)},
    {'battery_type': '9V', 'coordinates': (150, 250)},
    {'battery_type': 'D', 'coordinates': (200, 300)}
]

# Sort all detected batteries
for detection in detected_batteries:
    sort_battery(detection)

sock.close()  # Close the socket connection after completing the sorting


ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it